### 储能框架 基于规则集的

![规则集说明图](./规则集-第一版.png)

#### 输入的变量：
#### 下一个步长的P_pv，P_load，C_out，C_pv
#### 当前时间步长的SOC，SOC_max，SOC_min，P_bat_in_max，P_bat_out_max

In [5]:
mode_dict={1:"光伏>负荷，余电上网",2:"光伏>负荷，余电充进电池",3:"光伏>负荷，余电先充进电池，再上网",4:"光伏>负荷，余电上网",
               5:"负荷>光伏，外网电充进电池，负荷不足电网供电",6:"负荷>光伏，负荷不足由电网供电",
               7:"负荷>光伏，电池放电给负荷",8:"负荷>光伏，电池放电给负荷，剩余负荷不足由电网供电",
               9:"负荷>光伏，外网电充进电池，负荷不足由电网供电",10:"负荷>光伏，负荷不足由电网供电"}

In [6]:
def schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max):
    p_bat=0
    p_上网=0
    p_下网=0
    if p_pv>p_load:
        p_res=p_pv-p_load
        if soc<soc_max:
            if c_pv>c_out:###################################情况1
                p_上网=p_res
                mode=1
            else:
                if p_bat_in_max>p_res:#########################################情况2
                    p_bat=-p_res##充电
                    mode=2
                else:#######################################情况3
                    p_bat=-p_bat_in_max##充电
                    p_上网=p_res-p_bat_in_max
                    mode=3
        else:###################################################情况4
            p_上网=p_res
            mode=4
    else:
        p_lack=p_load-p_pv
        if soc>soc_min:
            if c_pv>c_out:
                if soc<soc_max:###################################################情况5
                    p_bat=-p_bat_in_max##充电
                    p_下网=p_lack+p_bat_in_max
                    mode=5
                else:###################################################情况6
                    p_下网=p_lack
                    mode=6
            else:
                if p_bat_out_max>p_lack:###################################################情况7
                    p_bat=p_lack##放电
                    mode=7
                else:###################################################情况8
                    p_bat=p_bat_out_max##放电
                    p_下网=p_lack-p_bat_out_max
                    mode=8
        else:
            if c_pv>c_out:###################################################情况9
                p_bat=-p_bat_in_max##充电
                p_下网=p_lack+p_bat_in_max
                mode=9
            else:###################################################情况10
                p_下网=p_lack
                mode=10
    return p_bat,p_上网,p_下网,mode

In [7]:
p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max=90,10,0.888,0.4,67.5,95,5,20,20

In [11]:
schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)

(-20, 60, 0, 3)